---
# Predicting SNCF wait times
---
In this notebook, we try to build a pipeline and framework in order to predict wait times for trains. 

We start by importing the data and splitting it. We then remove outliers and encoding categorical variables. 

Then, we perform hyperparameter optimization and train a simple XGBoost model accordingly. After this, we can export the results to make a submission on the challenge data website. 

Keep in mind that this notebook is only the tip of the iceberg : we have built libraries to preprocess features and test different models (they are accessible in the models.py and features.py modules). After trial and error, this is the best model that we can come up with.

## Importing packages and data

In [1]:
import numpy as np
import polars as pl
import polars.selectors as cs
from utils import PathsData, import_data, split_data, plot_correlation
from features import FeaturesFrame
from models import Model, ModelEnum, ParamGridEnum

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor


x_validation = import_data(PathsData.X_TEST.value) #! to use in the end to make a submission
x = import_data(PathsData.X_TRAIN.value)
y = import_data(PathsData.Y_TRAIN.value)


2025-04-05 23:06:30,911 - distributed.scheduler - WARNING - Worker failed to heartbeat for 2354s; attempting restart: <WorkerState 'tcp://127.0.0.1:51175', name: 3, status: running, memory: 0, processing: 0>
2025-04-05 23:06:30,954 - distributed.scheduler - WARNING - Worker failed to heartbeat for 2354s; attempting restart: <WorkerState 'tcp://127.0.0.1:51178', name: 0, status: running, memory: 0, processing: 0>
2025-04-05 23:06:30,956 - distributed.scheduler - WARNING - Worker failed to heartbeat for 2354s; attempting restart: <WorkerState 'tcp://127.0.0.1:51181', name: 2, status: running, memory: 0, processing: 0>
2025-04-05 23:06:30,958 - distributed.scheduler - WARNING - Worker failed to heartbeat for 2354s; attempting restart: <WorkerState 'tcp://127.0.0.1:51184', name: 1, status: running, memory: 0, processing: 0>
2025-04-05 23:06:32,482 - distributed.nanny - WARNING - Restarting worker
2025-04-05 23:06:32,503 - distributed.nanny - WARNING - Restarting worker
2025-04-05 23:06:32,

## Removing outliers
After testing different methods, we have found that the one that yields the best results is simply based on a simple quantile exclusion.
We have also tested different values to understand which is the best quantile to exclude. 

We start by visualizing the outliers with boxplots.

In [2]:
# to_plot = x.with_columns(y).select(cs.numeric())

# fig = make_subplots(rows=3, cols=3, subplot_titles=[c for c in to_plot.columns])

# fig.append_trace(go.Box(y=np.array(to_plot.select(to_plot.columns[0]).to_series().to_list()), boxpoints='all'), row=1, col=1)
# fig.append_trace(go.Box(y=np.array(to_plot.select(to_plot.columns[1]).to_series().to_list())), row=1, col=2)
# fig.append_trace(go.Box(y=np.array(to_plot.select(to_plot.columns[2]).to_series().to_list())), row=1, col=3)
# fig.append_trace(go.Box(y=np.array(to_plot.select(to_plot.columns[3]).to_series().to_list())), row=2, col=1)
# fig.append_trace(go.Box(y=np.array(to_plot.select(to_plot.columns[4]).to_series().to_list())), row=2, col=2)
# fig.append_trace(go.Box(y=np.array(to_plot.select(to_plot.columns[5]).to_series().to_list())), row=2, col=3)
# fig.append_trace(go.Box(y=np.array(to_plot.select(to_plot.columns[6]).to_series().to_list())), row=3, col=1)
# fig.append_trace(go.Box(y=np.array(to_plot.select(to_plot.columns[7]).to_series().to_list())), row=3, col=2)

# fig.update_layout(height=1000, width=1000, title_text="Outliers")

In [10]:

#remove outliers:
x = x.with_columns(y).filter(pl.col('p0q0').ge(pl.col('p0q0').quantile(0.05)) & pl.col('p0q0').le(pl.col('p0q0').quantile(0.95)))
y = x.select('p0q0')
x = x.drop('p0q0')

x_train, x_test, y_train, y_test = split_data(x=x, y=y)

We then encode the data using a frequency encoder

In [15]:
test_train = FeaturesFrame(x_train).encode_label(encoder='label').select(cs.numeric())#.add_feature_square().scale_standard() #.add_feature_interactions() #.add_feature_square() #.scale_standard(set="train")
test_test = FeaturesFrame(x_test).encode_label(encoder='label').select(cs.numeric()) #.add_feature_square().scale_standard(set='test', train_scaler=scaler) #.add_feature_interactions() #.add_feature_square() #.scale_standard(set="test", train_scaler=scaler)

And run a first test with the XGBoost model

In [16]:
model = Model(model_enum=ModelEnum.XGBoost)
model_fit = model.fit(test_train.to_numpy(), y_train=y_train.to_numpy())
model.evaluate(X_test=test_test.to_numpy(), y_test=y_test.to_numpy())

Model Evaluation - MSE: 0.5352, R2: 0.3688, MAE: 0.5525


(0.5351877907598194, 0.3688269234678675, 0.5525199492175347)

Finally, we run a grid search to find the optimal parameters

In [ ]:
model = Model(model_enum=ModelEnum.XGBoost)
params_opti_xgb = model.grid_search(test_train.to_pandas(), y_train=y_train.to_pandas(), param_grid = ParamGridEnum.XGBoost.value)

Fitting 3 folds for each of 108 candidates, totalling 324 fits


Then we train our model again with these params and print the results

In [ ]:
mod = XGBRegressor().set_params(**params_opti_xgb)
mod_fit = mod.fit(test_train.to_numpy(),y_train.to_numpy())
preds = mod.predict(X=test_test.to_numpy())
mse = mean_squared_error(y_test, preds)
r2 = r2_score(y_test, preds)
mae = mean_absolute_error(y_test,preds)
print(f"Model Evaluation - MSE: {mse:.4f}, R2: {r2:.4f}, MAE: {mae:.4f}")

Model Evaluation - MSE: 0.5215, R2: 0.3850, MAE: 0.5430


Making a submission:

In [22]:
def make_submission(number: int, model: XGBRegressor):
    predictions_validation = (
        model
        .predict(
            FeaturesFrame(x_validation)
            .encode_label(encoder='frequency')
            .drop('train', 'gare', 'date').to_numpy()
        )
    )
    (
        pl.DataFrame(predictions_validation)
        .rename({"column_0":"y_test"})
        .with_row_index()
        .write_csv(fr"C:\Users\faune\challenge-data-203\submissions\submission{number}.csv", separator=",")
    )
    print(f"Successfully saved submission number {number}")
    return None

In [23]:
make_submission(9, model=mod)

Successfully saved submission number 9
